In [0]:
import keras
from keras.layers import Dense,Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten,Concatenate,Input,Lambda
from keras.models import Sequential
import tensorflow as tf
import numpy as np 
import random
import matplotlib.pyplot as plt
from keras.models import Model
from keras import backend as K
import numpy as np
import glob
from sklearn.preprocessing import LabelBinarizer
import cv2
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt
from itertools import permutations
import seaborn as sns
from keras.datasets import mnist
from sklearn.manifold import TSNE
from sklearn.svm import SVC

In [0]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [8]:
x_test.shape

(10000, 28, 28)

In [0]:
def pairs_generation(x,y):   #Function to generate positive and negative pairs
  num_classes=max(y)+1       # Num_classes contain the number of classes
  pairs=[]                   # Initialize an empty array to store the pairs 
  labels=[]                  # Initialize an empty array to store the labels
  digit_indices=[np.where(y==i)[0] for i in range(num_classes)]    # used to create a list of indices of the images
  for idx in range(len(x)):                    # iterate though the entire dataset
    x1=x[idx]                                  # pick a sample
    label=y[idx]                               # pick a label of that sample
    idx2=random.choice(digit_indices[label])   # randomly pick the indice of the same sample picked
    x2=x[idx2]                                 # assign it to x2
    pairs+=[[x1,x2]]                           # contains identical pairs
    labels+=[1]                                # Labeled as 1 (Similar) 

    label2=random.randint(0,num_classes-1)    # Instantiate label2 to be a number between 0 and num_classes-1 
    while label2==label:                     # if Label2 is identical to Label1 we will pick from num_classes-1
      label2=random.randint(0,num_classes-1)   
    idx2=random.choice(digit_indices[label2])  # randomly choose an idx such that it belongs to label2 which is differnet than label1
    x2=x[idx2]                                 # Dissimilar pairs Genarated
    pairs+=[[x1,x2]]
    labels+=[0]
  return np.array(pairs),np.array(labels)
    

In [0]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)     #Some reshapping for compatibility

In [0]:
train_pairs,train_labels=pairs_generation(x_train,y_train)
test_pairs,test_labels=pairs_generation(x_test,y_test)

In [0]:
#Eucledian Distance Function  #Imported From Keras Documentation 
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


In [0]:
input=Input((28,28,1))                                               ###
x=keras.layers.Conv2D(32,3,strides=1,activation='relu')(input)      ####
x=keras.layers.MaxPool2D(pool_size=(2,2))(x)                         #####
x=keras.layers.Flatten()(x)                                            #####
dense=keras.layers.Dense(128)(x)                                        ##########
model=keras.models.Model(input,dense)                         ################ Base Network TO be Shared ############

inpu1=Input((28,28,1))                                         #Sharing of Base Network
inpu2=Input((28,28,1))
md1=model(inpu1)
md2=model(inpu2)

concat=keras.layers.concatenate([md1,md2])                   #Concatenating the two networks
DL=keras.layers.Dense(1,activation='sigmoid')(concat)        #Adding a final layer       

FM=Model(inputs=[inpu1,inpu2],outputs=DL)


In [0]:
FM.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

Epoch 1/10
120000/120000 [==============================] - 126s 1ms/step - loss: 1.1925 - accuracy: 0.4957
Epoch 2/10
120000/120000 [==============================] - 124s 1ms/step - loss: 0.6934 - accuracy: 0.4998
Epoch 3/10
120000/120000 [==============================] - 124s 1ms/step - loss: 0.6933 - accuracy: 0.5009
Epoch 4/10
120000/120000 [==============================] - 124s 1ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 5/10
120000/120000 [==============================] - 124s 1ms/step - loss: 0.6932 - accuracy: 0.4985
Epoch 6/10
120000/120000 [==============================] - 124s 1ms/step - loss: 0.6932 - accuracy: 0.4992
Epoch 7/10
120000/120000 [==============================] - 125s 1ms/step - loss: 0.6932 - accuracy: 0.4982
Epoch 8/10
120000/120000 [==============================] - 125s 1ms/step - loss: 0.6932 - accuracy: 0.4985
Epoch 9/10
120000/120000 [==============================] - 125s 1ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 10/10
120000/120000 [=

In [0]:
input=Input((28,28,1))                                               ###
x=keras.layers.Conv2D(32,3,strides=1,activation='relu')(input)      ####
x=keras.layers.MaxPool2D(pool_size=(2,2))(x)                         #####
x=keras.layers.Flatten()(x)                                            #####
dense=keras.layers.Dense(128)(x)                                        ##########
model=keras.models.Model(input,dense)                         ################ Base Network TO be Shared ############

inpu1=Input((28,28,1))                                         #Sharing of Base Network
inpu2=Input((28,28,1))
md1=model(inpu1)
md2=model(inpu2)

merged=keras.layers.Lambda(euclidean_distance)([md1,md2])                   #Concatenating the two networks using the Lambda layer to combine with eucledian_distance
DL=keras.layers.Dense(1,activation='sigmoid')(merged)        #Adding a final layer       

FM=Model(inputs=[inpu1,inpu2],outputs=DL)


In [82]:
FM.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')
FM.fit([train_pairs[:,0],train_pairs[:,1]],train_labels[:],epochs=10)

Epoch 1/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.7486 - accuracy: 0.4943
Epoch 2/10
120000/120000 [==============================] - 126s 1ms/step - loss: 0.6747 - accuracy: 0.5244
Epoch 3/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.3228 - accuracy: 0.9044
Epoch 4/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.1766 - accuracy: 0.9529
Epoch 5/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.1240 - accuracy: 0.9647
Epoch 6/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.0985 - accuracy: 0.9701
Epoch 7/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.0832 - accuracy: 0.9748
Epoch 8/10
120000/120000 [==============================] - 127s 1ms/step - loss: 0.0733 - accuracy: 0.9772
Epoch 9/10
120000/120000 [==============================] - 126s 1ms/step - loss: 0.0643 - accuracy: 0.9805
Epoch 10/10
120000/120000 [=